In [1]:
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle

In [2]:
URLs = ['https://en.wikipedia.org/wiki/List_of_animal_names',
      'https://en.wikipedia.org/wiki/List_of_fish_common_names',
      'https://en.wikipedia.org/wiki/List_of_trees_and_shrubs_by_taxonomic_family',
      'https://en.wikipedia.org/wiki/List_of_birds_by_common_name']

URL = 'https://en.wikipedia.org/wiki/List_of_fish_common_names'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
WikiRefs = soup.find_all('a')
# Create links 
WikiLinks = ['https://en.wikipedia.org' + pages.get('href') for pages in WikiRefs 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]
            
# Remove duplicates            
WikiLinks = list(set(WikiLinks))
len(WikiLinks)

1274

In [3]:
WikiSubLinks = []

for WikiLink in WikiLinks:

    URL = WikiLink
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find all wikiparts
    WikiRefs = soup.find_all('a')
    # Create links 
    WikiSubLinks += ['https://en.wikipedia.org' + pages.get('href') for pages in WikiRefs 
                           if pages.get('href') != None 
                           if pages.get('href').startswith('/wiki/')]


WikiSubLinks = list(set(WikiSubLinks))
len(WikiSubLinks)

49747

In [4]:
WikiLinks += WikiSubLinks
# Remove duplicates            
WikiLinks = list(set(WikiLinks))
len(WikiLinks)

49747

In [5]:
# Init Dict
Data = collections.defaultdict(list)

# Known chapter titles
KnownDescriptionList = ['Physical characteristics', 'Description and morphology',
                        'Description', 'General description',
                        'Appearance',
                        'Characteristics']

# Lower them
KnownDescriptionListLower =[i.lower() for i in KnownDescriptionList]

# Removes references in text
ReferenceRemover = '\[\d*\]'

for Count, WikiLink in enumerate(WikiLinks):
    
    if Count % 1000 == 0:
        print(Count)

    URL = WikiLink

    try:
        Response = requests.get(URL, timeout=5)
        Soup = BeautifulSoup(Response.content, "html.parser")

        # Check if the link contains description data
        Contents = [i.text.replace('[edit]', '') for i in Soup.find_all('h2')]
        if all(s not in Contents for s in KnownDescriptionList) == False:

            # Find Chapters
            for Tag in Soup.find_all('h2')[1:]:


                # Get species name
                Species = Soup.title.string\
                                    .split(' - ')[0]\
                                    .rstrip(' ')

                # Clean Chapter
                Chapter = Tag.text.strip().replace('[edit]', '')

                # Find all Text
                for Item in Tag.find_next_siblings('p'):
                    # Check if Chapter is Description or similar
                    if Chapter.lower() in KnownDescriptionListLower:
                        # Check if text belongs to current chapter
                        if Chapter in Item.find_previous_siblings('h2')[0].text.strip():
                            # Clean Text
                            CleanText = re.sub(ReferenceRemover, '', Item.text)
                            # Add to dict
                            Data[Species].append(tuple([1, CleanText]))

                    else:
                        # Check if text belongs to current chapter
                        if Chapter in Item.find_previous_siblings('h2')[0].text.strip():
                            # Clean Text
                            CleanText = re.sub(ReferenceRemover, '', Item.text)
                            # Add to dict
                            Data[Species].append(tuple([0, CleanText]))
    except:
        # Continue if timeout
        continue



0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [6]:
with open('../data/processed/dataWIKI_fish.pkl', 'wb') as f:
    pickle.dump(Data, f)

In [7]:
len(Data.keys())

3665

In [ ]:
Data['Asclepias incarnata']